In [1]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
from albumentations import HorizontalFlip, VerticalFlip, Rotate
from matplotlib import pyplot as plt
import pywt
import pandas as pd
import ast

In [2]:
dirr = "C:\\Users\\subha\\OneDrive\\Documents\\Thyroid ultrasound image_lite"    #data path
dirr1 = "C:\\Users\\subha\\OneDrive\\Documents\\th_cancer\\image"      #Final augmented images save at this path
dirr2 = "C:\\Users\\subha\\OneDrive\\Documents\\th_cancer\\mask"       #Mask is saved at this path
listt = []
path = []
label_path = []
f_data_path = []
for i in range(100):
    if (i==36 or i==53):
        listt.append([])
        path.append([])
        label_path.append([])
        f_data_path.append([])
        pass
    else:
        listt.append([])
        path.append([])
        label_path.append([])
        f_data_path.append([])
        n = str(i+1)
        p = n + "_"
        n= n + ".xml"
        path1 = os.path.join(dirr, n)
        pt1 = pd.read_xml(path1)
        ptt1 = pt1.svg.dropna()
        
        for j in range(ptt1.count()):
            v = p + str(j+1) +".jpg"
            t = p + str(j+1) +".jpg"
            path2 = os.path.join(dirr, v)
            path3 = os.path.join(dirr2, t)
            path4 = os.path.join(dirr1, v)
            path[i].append(path2)
            label_path[i].append(path3)
            f_data_path[i].append(path4)
            str1 = ptt1.iloc[j]
            list1 = ast.literal_eval(str1)
            pt = list1[0].get("points")
            listt[i].append(pt)

In [3]:
def mask_img(img, pt):
    blank_image = np.zeros(shape=[img.shape[0], img.shape[1]], dtype=np.uint8)
    color = (1)
    thickness = 1
    for i in range(len(pt)-1):
        cv2.line(blank_image, (pt[i].get("x"),pt[i].get("y")), (pt[i+1].get("x"),pt[i+1].get("y")),color, thickness)
    for i in range (blank_image.shape[0]):
        indices  = [index for (index, item) in enumerate(blank_image[i]) if item == 1]
        if (len(indices)!= 0):
            for j in range(indices[0],indices[-1]):
                blank_image[i][j]=250
    return blank_image

In [4]:
    
def ft(img):                                   #fourier transformation
    
    dft = cv2.dft(np.float32(img), flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)
    magnitude_spectrum = 20 * np.log(cv2.magnitude(dft_shift[:, :, 0], dft_shift[:, :, 1]))
    rows, cols = img.shape
    crow, ccol = int(rows / 2), int(cols / 2)
    mask = np.ones((rows, cols, 2), np.uint8)
    r = 7
    center = [crow, ccol]
    x, y = np.ogrid[:rows, :cols]
    mask_area = (x - center[0]) ** 2 + (y - center[1]) ** 2 <= r*r
    mask[mask_area] = 0
    fshift = dft_shift * mask

    fshift_mask_mag = 1000 * np.log(cv2.magnitude(fshift[:, :, 0], fshift[:, :, 1]))

    f_ishift = np.fft.ifftshift(fshift)
    img_back = cv2.idft(f_ishift)
    img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])

    norm_img = np.zeros((rows, cols))
    
    ft_img = cv2.normalize(img_back,  norm_img, 0, 255, cv2.NORM_MINMAX)
    return ft_img

In [5]:
def wt(img):                                            #wavelet transformation
    coeffs2=pywt.dwt2(img,'haar')
    LL, (LH, HL, HH) = coeffs2
    LL=np.uint8(LL)
    LH=np.uint8(LH)
    HL=np.uint8(HL)
    HH=np.uint8(HH)
    coffes2=LL,(LH,HL,HH)
    wt_img = pywt.idwt2(coffes2,'haar')
    rows, cols = img.shape
    norm_img = np.zeros((rows, cols))
    wt_img = cv2.normalize(wt_img,  norm_img, 0, 255, cv2.NORM_MINMAX)
    
    return wt_img

In [6]:
def bf(img):                                            #bilateral filter2
    y_img = cv2.bilateralFilter(img,16,50,80)
    rows, cols = img.shape
    norm_img = np.zeros((rows, cols))
    bf_img = cv2.normalize(y_img,  norm_img, 0, 255, cv2.NORM_MINMAX)
    return bf_img

In [7]:
def bf2(img):                                         #bilateral filter2
    y_img = cv2.bilateralFilter(img,20,5,8)
    rows, cols = img.shape
    norm_img = np.zeros((rows, cols))
    bf_img = cv2.normalize(y_img,  norm_img, 0, 255, cv2.NORM_MINMAX)
    return bf_img

In [8]:
list_data = []
list_label = []
for i in range(100):
    if (len(listt[i]) != 0):
        for j in range(len(listt[i])):
            img = cv2.imread(path[i][j], 0)
            
            wt_img= wt(img)
            ft_img= ft(img)
            bf_img= bf(img)
            wt_img = np.uint8(wt_img)
            wt_img = bf2(wt_img)
            
            f_img = []
            for k in range (len(wt_img)):
                f_img.append([])
                for l in range (len(wt_img[k])):   #Final image have  80% bilateral filter 15% fourier transfored image and 5% wavelet transfored image
                    f_img[k].append(np.round((0.05*wt_img[k][l]+.15*ft_img[k][l]+0.8*bf_img[k][l])))
            img = np.uint8(f_img)
            
            mask = mask_img(img, listt[i][j])
            cv2.imwrite(f_data_path[i][j],img)
            cv2.imwrite(label_path[i][j],mask)
            list_data.append(path[i][j])
            list_label.append(label_path[i][j])

C:\Users\subha\AppData\Local\Temp\ipykernel_31036\3724126425.py:16: RuntimeWarning: divide by zero encountered in log
  fshift_mask_mag = 1000 * np.log(cv2.magnitude(fshift[:, :, 0], fshift[:, :, 1]))


In [9]:
def load_data(path):
    x_train = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y_train = sorted(glob(os.path.join(path, "mask", "*.jpg")))
    
    return x_train, y_train



In [10]:
def augment_data(image, mask, save_path, augment = True):
    size = (256,128)
    # image crop shape
    x1 = 58
    x2 = 250
    y1 = 86
    y2 = 450
    for idx, (X, Y) in tqdm(enumerate(zip(image, mask)), total = len(image)):
        name = X.split("\\")[-1].split(".")[0]
        X = cv2.imread(X, cv2.IMREAD_COLOR)
        Y = cv2.imread(Y, cv2.IMREAD_COLOR)
        if augment == True:
            pass
        else:
            x = [X[58:250,86:450]]
            y = [Y[58:250,86:450]]
        index = 0
        for i,m in zip(x,y):
            i = cv2.resize(i, size)
            m = cv2.resize(m, size)
            tem_img_name = f"{name}.png"
            tem_mask_name = f"{name}.png"
            
            img_path = os.path.join(save_path, "image", tem_img_name)
            mask_path = os.path.join(save_path, "mask", tem_mask_name)
            
            cv2.imwrite(img_path, i)
            cv2.imwrite(mask_path, m)
            
            index+=1
        

In [11]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [12]:
if __name__ == "__main__":
    path = "C:\\Users\\subha\\OneDrive\\Documents\\th_cancer"
    x_train, y_train = load_data(path)
    print(len(x_train),len(y_train))
    create_dir("C:/Users/subha/OneDrive/Documents/th_cancer_data/new1_image/train/image")
    create_dir("C:/Users/subha/OneDrive/Documents/th_cancer_data/new1_image/train/mask")
    path = "C:/Users/subha/OneDrive/Documents/th_cancer_data/new1_image/train"
    augment_data(x_train, y_train, path, augment = False)

119 119


100%|███████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 120.30it/s]
